# Nonparametric inference changes in spm1d v0.4.50 (2nd bug)

(2025-06-06)

In repairing the [first bug](nonparam_bug_fix.ipynb) a second bug was discovered.

The bug affects both the Python and MATLAB versions of **spm1d** for all versions prior to v0.4.50.

The bug affects **only 0D data analysis** and **only two-tailed inference** for the following procedures:

- `spm1d.stats.nonparam.ttest`
- `spm1d.stats.nonparam.ttest_paired`
- `spm1d.stats.nonparam.ttest2`
- `spm1d.stats.nonparam.regress`
- `spm1d.stats.nonparam.ci_onesample`
- `spm1d.stats.nonparam.ci_pairedsample`
- `spm1d.stats.nonparam.ci_twosample`

The bug applied a double-correction for two-tailed inference when calculating critical threshold, thereby yielding overly conservative thresholds. The p-values from these procedures are unaffected. Thus the bug affects only cases where the critical threshold is of primary concern. As such the practical implications of this bug are expected to be limited to 0D confidence interval results.

Below is a brief description of the bug and a demonstration of its resolution in spm1d v0.4.50.

<br>
<br>

___

## Preliminaries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import spm1d

Before starting verify that you have **spm1d** version 0.4.50 or later:

In [2]:
print( spm1d.__version__ )

0.4.50


Note that the `spm1d.stats.nonparam` subpackage has been rewritten. If you wish to replicate previous versions' results please use the `spm1d.stats.nonparam_old` package which retains the original code.

In [3]:
import spm1d.stats.nonparam_old

<br>
<br>

___

## Bug example

The example below shows that the old procedure a critical threshold of 2.57, considerably higher than the 2.20 and 2.23 yielded by the parametric and new nonparametric procedures.

In [4]:
# create data:
y0         = np.array([5.7, 8.4, 6.0, 6.4, 5.5])
y1         = np.array([5.3, 8.2, 5.5, 6.1, 5.6])
np.random.seed(0)
y0         = np.random.randn(12)
y1         = np.random.randn(12)


# conduct parametric and nonparametric tests:
np.random.seed(0)
alpha      = 0.05
two_tailed = True
niter      = -1
ti         = spm1d.stats.ttest_paired(y1, y0).inference(alpha, two_tailed=two_tailed)
tni        = spm1d.stats.nonparam.ttest_paired(y1, y0).inference(alpha, two_tailed=two_tailed, iterations=niter)
tnio       = spm1d.stats.nonparam_old.ttest_paired(y1, y0).inference(alpha, two_tailed=two_tailed, iterations=niter)


print( 'Critical thresholds:')
print( f'   Parametric:           {ti.zstar:.5f}')
print( f'   Nonparametric:        {tni.zstar[1]:.5f}')
print( f'   Nonparametric (old):  {tnio.zstar[1]:.5f}')



_inference_0d_twotailed
Critical thresholds:
   Parametric:           2.20099
   Nonparametric:        2.22788
   Nonparametric (old):  2.57330


The bug can be rectified in the old version simply by multiplying the desired alpha by two:

In [5]:
tnio       = spm1d.stats.nonparam_old.ttest_paired(y1, y0).inference(2*alpha, two_tailed=two_tailed, iterations=niter)

print( f'   Nonparametric (old corrected):  {tnio.zstar[1]:.5f}')

   Nonparametric (old corrected):  2.22788
